
## Processamento de Linguagem Natural
 - Comunicação Homem-Máquina
     - input natural --[NLP]--> output natural
     
## Corretor Ortográfico
**INPUT -> ALGORITMO -> OUTPUT**
Algoritmo:
- Criar Vocabulário [DATABASE]
*CORPUS* -> Base de dados, corpo com diversos textos, conjunto de documentos
documentos -> Elementos do *Corpus*
Token -> Elementos do vetor, separar a string em elementos diferentes (estes elementos são os tokens)


## Documentations:
[Natural Language Toolkit (NLTK)](nltk.org)

In [1]:
# Imports:

import nltk
import re

In [2]:
with open('data/artigos.txt', 'r', encoding="utf8") as f: #f = file
    artigos=f.read()

In [3]:
#nlkt.download('punkt') # to use tokenize

# Limpando os dados:
- Retirar pontuações;
- Limpar palavras com letras maiúsculas;
- Limpar palavras repetidas.
**BONUS:**
- sorting values para facilitar o algoritmo de pesquisa por meio de pesquisa binária;

**Este Corpus é interessante para o NLP?**
- Ele tem palavras o suficiente?
- Ele tem muitas palavras repetidas?

In [4]:
# Retirar pontuações

def separa_palavras(lista_tokens):
    lista_retorno=[]
    for token in lista_tokens:
        if token.isalpha():
            lista_retorno.append(token)
    return lista_retorno

#import sys
#sys.setrecursionlimit(len(lista_tokens))
def retira_pontuacoes(lista_tokens:list, lista_retorno:list=[], index:int=0):
    """Função recursiva para retirar pontuações, checa se ainda estamos dentro da lista, caso sim,
    checa se a palavra no índice index possui apenas conjuntos alpha, se sim, adicionamos esta palavra à
    lista de retorno lista_retorno, retornamos à função atualizando a lista de retorno e índice.
    caso o índice seja maior que o "tamanho da lista de tokens - 1", retornamos lista final.
    """
    if index < len(lista_tokens):
        if lista_tokens[index].isalpha(): 
            lista_retorno.append(lista_tokens[index])
        return retira_pontuacoes(lista_tokens, lista_retorno, (index+1))
    return lista_retorno

In [5]:
# Descobrir quantidade de PALAVRAS [sem pontuação]

lista_tokens=nltk.tokenize.word_tokenize(artigos)
lista_palavras= separa_palavras(lista_tokens)

In [6]:
print("O número de palavras dentro da lista não tratada é: {}".format(len(lista_tokens))) # Número de tokens NÃO tratados dentro da lista [PONTUAÇÕES]
print("O número de palavras dentro da lista tratada é: {}".format(len(lista_palavras))) # Número de tokens tratados dentro da lista [PONTUAÇÕES]

O número de palavras dentro da lista não tratada é: 515827
O número de palavras dentro da lista tratada é: 403031


In [28]:
# Quantidade de palavras repetidas
# Cuidar letras maiusculas! Passar tudo .lower()

def tratar_repeticoes(lista_tokens):
    lista_retorno=[]
    for token in lista_tokens:
        if token.lower() not in lista_retorno:
            lista_retorno.append(token.lower())
    return lista_retorno

# tokens_tratados=tratar_repeticoes(lista_palavras)

# Outra função interessante:
def tratar_normalizacao(lista_tokens):
    # Transforma todas as palavras da lista em minusculo
    lista_retorno=[]
    for token in lista_tokens:
        lista_retorno.append(token.lower())
    return lista_retorno

tokens_tratados3=tratar_normalizacao(lista_palavras)
tokens_tratados= set(tratar_normalizacao(lista_palavras))

In [29]:
print('Listas de palavras sem repetições, em minusculo:')
print(len(tokens_tratados))
print('Listas de palavras com repetições, em minusculo:')
print(len(tokens_tratados3))

Listas de palavras sem repetições, em minusculo:
18464
Listas de palavras com repetições, em minusculo:
403031


# Desenvolvendo o Corretor
DB = tokens_tratados3

### Letras => Operação (inserção) => Retorno

# GERADOR DE PALAVRAS

In [153]:
example='eme'

def gerador_palavras(palavra:str):
    fatias= []
    for i in range(len(palavra)+1):
        fatias.append((palavra[:i], palavra[i:]))
        palavras_geradas=insere_letras(fatias)
        
        palavras_geradas_D= deleta_letras(fatias)
        
        palavras_geradas_S= substitui_letras(fatias)
        for element in palavras_geradas_D:
            palavras_geradas.append(element)
        for element in palavras_geradas_S:
            palavras_geradas.append(element)
    return palavras_geradas

# INSERÇÃO

In [154]:
def insere_letras(fatias:list):
    novas_palavras=[]
    letras= 'abcdefghijklmnopqrstuvwxyzáàâãéèêẽiíìĩîóòõôùúũûç'
    for L, R in fatias:
        for letra in letras:
            novas_palavras.append(L + letra + R)
    return novas_palavras


# DELEÇÃO

In [155]:
def deleta_letras(fatias:list):
    novas_palavras=[]
    for L, R in fatias:
            novas_palavras.append(L+ R[1:])
    return novas_palavras

# SUBSTITUIÇÃO

In [156]:
def substitui_letras(fatias:list):
    novas_palavras=[]
    letras= 'abcdefghijklmnopqrstuvwxyzáàâãéèêẽiíìĩîóòõôùúũûç'
    for L, R in fatias:
        for letra in letras:
            novas_palavras.append(L+ letra + R[1:])
    return(novas_palavras)

# ALGORITMO CORRETOR

In [157]:
def corretor(palavra):
    palavras_geradas=gerador_palavras(palavra)
    palavra_correta = max(palavras_geradas, key=probabilidade)
    return palavra_correta

frequencia = nltk.FreqDist(tokens_tratados3) # LISTA COM REPETIÇÕES

def probabilidade(palavra_gerada):
    """frequencia_palavra/total_de_palavras"""
    prob = frequencia[palavra_gerada]/len(tokens_tratados3)
    return prob

In [158]:
corretor(example)

'em'

# FUNÇÃO DE TESTE:

In [163]:
def cria_dados_teste(nome_arquivo):
    lista_dados_teste=[]
    #ler arquivo
    f = open(nome_arquivo, 'r')
    for linha in f:
        correta, errada = linha.split() #Separar as palavras
        lista_dados_teste.append((correta, errada)) #append ((correta, errada), (correta1, errada1))
    f.close()
    return lista_dados_teste

def test(testes):
    acertou=0
    for correta, errada in testes:
        corrigida= corretor(errada)
        if corrigida == correta:
            acertou+=1
            #print('CERTO! INPUT: {} -> OUTPUT: {} -> RIGHT: {}'.format(errada, corrigida, correta))
        else:
            print('ERRADO! INPUT: {} -> OUTPUT: {} -> RIGHT: {}'.format(errada, corrigida, correta))
    taxa_acerto= float(acertou/len(testes))
    print('taxa de acerto: {}%'.format(round((taxa_acerto*100) ,4)))# de {} palavras'.format(round((taxa_acerto*100), 4), len(testes)))

In [165]:
test_data=cria_dados_teste('./data/test_data.txt')
test(test_data)

ERRADO! INPUT: eme -> OUTPUT: em -> RIGHT: ele
ERRADO! INPUT: noâ -> OUTPUT: no -> RIGHT: nos
ERRADO! INPUT: teb -> OUTPUT: tem -> RIGHT: ter
ERRADO! INPUT: âem -> OUTPUT: em -> RIGHT: bem
ERRADO! INPUT: serr -> OUTPUT: ser -> RIGHT: será
ERRADO! INPUT: eû -> OUTPUT: e -> RIGHT: eu
ERRADO! INPUT: jé -> OUTPUT: é -> RIGHT: já
ERRADO! INPUT: dms -> OUTPUT: dos -> RIGHT: das
ERRADO! INPUT: cava -> OUTPUT: java -> RIGHT: cada
ERRADO! INPUT: nossk -> OUTPUT: nosso -> RIGHT: nossa
ERRADO! INPUT: ìeu -> OUTPUT: seu -> RIGHT: eu
ERRADO! INPUT: qelay -> OUTPUT: delay -> RIGHT: ela
ERRADO! INPUT: dtilizacr -> OUTPUT: adtilizacr -> RIGHT: utilizar
ERRADO! INPUT: bprojõto -> OUTPUT: abprojõto -> RIGHT: projeto
ERRADO! INPUT: ysiteo -> OUTPUT: aysiteo -> RIGHT: site
ERRADO! INPUT: sõêm -> OUTPUT: asõêm -> RIGHT: sem
ERRADO! INPUT: peàli -> OUTPUT: apeàli -> RIGHT: pelo
ERRADO! INPUT: asuraó -> OUTPUT: aasuraó -> RIGHT: alura
ERRADO! INPUT: deiìa -> OUTPUT: deixa -> RIGHT: dia
ERRADO! INPUT: tuĩdoì 

In [ ]:
# OUTRA FORMA DE PENSAR:
## QUAIS PALAVRAS POSSUEM ESTAS LETRAS?
## CHECAR PROBABILIDADE DAS PALAVRAS
## RETORNAR A PALAVRA COM MAIOR PROBABILIDADE